In [1]:
# Import necessary libraries
import cv2
import mediapipe as mp
from deepface import DeepFace
import time
import os

In [2]:
# Configuration
MODEL_NAME = 'Facenet'
THRESHOLD = 0.4
IMAGE_DIR = "images"
CAPTURED_IMAGE = "captured.jpg"

In [3]:
# Initialize MediaPipe face detection
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

In [4]:
# Function to detect faces and draw bounding boxes
def run_face_detection():
    cap = cv2.VideoCapture(0)
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            image.flags.writeable = False
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image_rgb)

            image.flags.writeable = True
            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

            if results.detections:
                for detection in results.detections:
                    mp_drawing.draw_detection(image_bgr, detection)

            cv2.imshow('Face Detection', cv2.flip(image_bgr, 1))

            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()
    cv2.destroyAllWindows()

In [5]:
# Function to run MediaPipe FaceMesh
def run_face_mesh():
    cap = cv2.VideoCapture(0)
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                continue

            image.flags.writeable = False
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)

            image.flags.writeable = True
            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        image=image_bgr,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=drawing_spec)

            cv2.imshow('Face Mesh', cv2.flip(image_bgr, 1))

            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# Load all reference images into a labeled dictionary
def load_face_database():
    database = {"Kobe": [], "Freeman": []}
    for file in os.listdir(IMAGE_DIR):
        path = os.path.join(IMAGE_DIR, file)
        if "kobe" in file.lower():
            database["Kobe"].append(path)
        elif "freeman" in file.lower():
            database["Freeman"].append(path)
        elif "caleb" in file.lower():
            database["Caleb"].append(path)
    return database

In [7]:
# Compare a new image with all known identities
def identify_person(captured_img, database):
    distances = {}
    for person, images in database.items():
        total_distance = 0
        valid = 0
        for ref_img in images:
            try:
                result = DeepFace.verify(captured_img, ref_img, model_name=MODEL_NAME, enforce_detection=True)
                total_distance += result['distance']
                valid += 1
            except Exception as e:
                print(f"Error comparing with {ref_img}: {e}")
                continue
        if valid:
            avg = total_distance / valid
            distances[person] = avg

    if distances:
        best_match = min(distances, key=distances.get)
        if distances[best_match] < THRESHOLD:
            print(f"✅ Identified as {best_match} (avg distance: {distances[best_match]:.4f})")
        else:
            print(f"❌ Unknown face (min distance: {distances[best_match]:.4f})")
    else:
        print("⚠️ No face match found.")

In [8]:
# Capture a photo from webcam
def capture_image(filename):
    cap = cv2.VideoCapture(0)
    time.sleep(2)
    print("Press SPACE to capture...")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        cv2.imshow("Capture", frame)
        key = cv2.waitKey(1)

        if key == 32:
            cv2.imwrite(filename, frame)
            print(f"📸 Saved as {filename}")
            break
        elif key == 27:
            break
    cap.release()
    cv2.destroyAllWindows()
    return os.path.exists(filename)

In [9]:
# Main function
def main():
    print("Loading face database...")
    db = load_face_database()
    if capture_image(CAPTURED_IMAGE):
        identify_person(CAPTURED_IMAGE, db)

if __name__ == "__main__":
    # Choose which module to run
    # run_face_detection()
    # run_face_mesh()
    main()

Loading face database...


KeyError: 'Caleb'